# Tracing LangChain apps using Prompt flow & OpenTelemery

The tracing capability provided by Prompt flow is built on top of [OpenTelemetry](https://opentelemetry.io/) that gives you complete observability over your LLM applications. 
And there is already a rich set of OpenTelemetry [instrumentation packages](https://opentelemetry.io/ecosystem/registry/?language=python&component=instrumentation) available in OpenTelemetry Eco System. 

In this example we will demo how to use [opentelemetry-instrumentation-langchain](https://github.com/traceloop/openllmetry/tree/main/packages/opentelemetry-instrumentation-langchain) package provided by [Traceloop](https://www.traceloop.com/) to instrument [LangChain](https://python.langchain.com/docs/get_started/quickstart) apps.


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Trace `LangChain` applications and visualize the trace of your application in prompt flow.

## Requirements

To run this notebook example, please install required dependencies:

In [ ]:
%%capture --no-stderr
%pip install -r ./requirements.txt

## Start tracing LangChain using promptflow

Start trace using `promptflow.start_trace`, click the printed url to view the trace ui.

In [ ]:
from promptflow import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

By default, `opentelemetry-instrumentation-langchain` instrumentation logs prompts, completions, and embeddings to span attributes. This gives you a clear visibility into how your LLM application is working, and can make it easy to debug and evaluate the quality of the outputs.

In [ ]:
# enable langchain instrumentation
from opentelemetry.instrumentation.langchain import LangchainInstrumentor

instrumentor = LangchainInstrumentor()
if not instrumentor.is_instrumented_by_opentelemetry:
    instrumentor.instrument()

## Run a simple Lang Chain

Below is an example targeting an AzureOpenAI resource. Please configure you `API_KEY` using an [.env](.env) file, see [.env.example](.env.example).

In [ ]:
import os

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

if "OPENAI_API_KEY" not in os.environ:
    # load environment variables from .env file
    load_dotenv()

llm = AzureChatOpenAI(
    deployment_name=os.environ["CHAT_DEPLOYMENT_NAME"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_type="azure",
    openai_api_version="2023-07-01-preview",
    temperature=0,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

chain = LLMChain(llm=llm, prompt=prompt, output_key='metrics')
chain({"input": "What is ChatGPT?"})

You should be able to see traces of the chain in promptflow UI now.

# Next Steps

By now you've successfully tracing LLM calls in your app using prompt flow.

You can check out more examples:
- [Trace your flow](../../../flows/standard/basic-code-first/quickstart.ipynb): using promptflow @trace to structurally tracing your app and do evaluation on it with batch run.